In [1]:
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

Using TensorFlow backend.


In [2]:
!nvidia-smi

Wed Jul 18 12:43:46 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:02:00.0 Off |                  N/A |
| 56%   64C    P0    84W / 300W |      2MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers.core import Dense, Lambda, Flatten, Reshape, Dropout
from keras.layers.convolutional import Conv2D, UpSampling2D,Cropping2D,ZeroPadding2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate, Add
from keras.layers import Input
from keras.models import Sequential
from keras.layers.core import Activation
from keras import optimizers
from keras import losses
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#imported from Kaggle
#import numpy as np
quizzes = np.zeros((1000000, 81), np.int32)
solutions = np.zeros((1000000, 81), np.int32)
for i, line in enumerate(open('data/sudoku.csv', 'r').read().splitlines()[1:]):
    quiz, solution = line.split(",")
    for j, q_s in enumerate(zip(quiz, solution)):
        q, s = q_s
        quizzes[i, j] = q
        solutions[i, j] = s
             
x=quizzes #data
l=solutions #labels

quizzes = quizzes.reshape((-1, 9, 9))
solutions = solutions.reshape((-1, 9, 9))

#DNN input/output width
#input_width=(9,9)#len(x[0,:])#(9,9)
#output_width=(9,9)#len(l[0,:])#(9,9)

In [5]:
#print a few Sudokus
for quiz, solution in zip(quizzes[:1], solutions[:1]):
    print("Sudoku with erased positions:\n", quiz)
    print("\nSudoku with correct labels:\n",solution)

Sudoku with erased positions:
 [[0 0 4 3 0 0 2 0 9]
 [0 0 5 0 0 9 0 0 1]
 [0 7 0 0 6 0 0 4 3]
 [0 0 6 0 0 2 0 8 7]
 [1 9 0 0 0 7 4 0 0]
 [0 5 0 0 8 3 0 0 0]
 [6 0 0 0 0 0 1 0 5]
 [0 0 3 5 0 8 6 9 0]
 [0 4 2 9 1 0 3 0 0]]

Sudoku with correct labels:
 [[8 6 4 3 7 1 2 5 9]
 [3 2 5 8 4 9 7 6 1]
 [9 7 1 2 6 5 8 4 3]
 [4 3 6 1 9 2 5 8 7]
 [1 9 8 6 5 7 4 3 2]
 [2 5 7 4 8 3 9 1 6]
 [6 8 9 7 3 4 1 2 5]
 [7 1 3 5 2 8 6 9 4]
 [5 4 2 9 1 6 3 7 8]]


In [6]:
x=np.array(quizzes)
l=np.array(solutions)
print("l:",l[0])

x=x.reshape(1000000,9,9,1)
l=l.reshape(1000000,1,1,81)
print("l after reshape:",l[0])

#use small dataset for faster learning 
Nsubset=1000000#900000
x=x[0:Nsubset,:,:,:]
l=l[0:Nsubset,:,:,:]

x=x.reshape(Nsubset,9,9,1)
l=l.reshape(Nsubset,9,9,1)
#print("l after second reshape:",l[0],x[0])

l: [[8 6 4 3 7 1 2 5 9]
 [3 2 5 8 4 9 7 6 1]
 [9 7 1 2 6 5 8 4 3]
 [4 3 6 1 9 2 5 8 7]
 [1 9 8 6 5 7 4 3 2]
 [2 5 7 4 8 3 9 1 6]
 [6 8 9 7 3 4 1 2 5]
 [7 1 3 5 2 8 6 9 4]
 [5 4 2 9 1 6 3 7 8]]
l after reshape: [[[8 6 4 3 7 1 2 5 9 3 2 5 8 4 9 7 6 1 9 7 1 2 6 5 8 4 3 4 3 6 1 9 2 5 8 7
   1 9 8 6 5 7 4 3 2 2 5 7 4 8 3 9 1 6 6 8 9 7 3 4 1 2 5 7 1 3 5 2 8 6 9 4
   5 4 2 9 1 6 3 7 8]]]


In [7]:
#use one-hot coded representation
n=Nsubset
test=l[0:n,:] #it has all solutions with dim 9x9x1
print(test[0].shape)
test=test.reshape(n,81)
print('test after reshape:',test[0])


test_hot=np.zeros((n,81,10)) # it is 81x10

for i in range(0,n):
    temp=np.eye(10)[test[i,:]]      
    #print(temp)
    test_hot[i,:]=temp #one-hot coded solutions
    #print('test_hot:',test_hot[0])
    
#print('testhot:',test_hot[0])
l_oh=test_hot.reshape(n,9,9,10) #one-hot coded solution with shape 9x10 each

(9, 9, 1)
test after reshape: [8 6 4 3 7 1 2 5 9 3 2 5 8 4 9 7 6 1 9 7 1 2 6 5 8 4 3 4 3 6 1 9 2 5 8 7 1
 9 8 6 5 7 4 3 2 2 5 7 4 8 3 9 1 6 6 8 9 7 3 4 1 2 5 7 1 3 5 2 8 6 9 4 5 4
 2 9 1 6 3 7 8]


In [8]:
###################################################################################
#Parameters
optimizer=optimizers.Adam(lr=0.001)
input_shape = (9,9,1)
blocks=8

input_layer = Input(shape = (9,9,1))
l1 = Conv2D(256, kernel_size=(3, 3), strides=(1, 1),padding='same',activation='relu',kernel_initializer='glorot_uniform')(input_layer)
l_branch=l1
for j in range(0,blocks):
    branch1 = (Conv2D(128, kernel_size=(1, 9),padding='same', activation='linear'))(l_branch)
    branch2 = (Conv2D(128, kernel_size=(9, 1),padding='same', activation='linear'))(l_branch)

    l_inc=Concatenate()([branch1,branch2])
    l_shortcut=l_branch
    l_branch = Add()([l_inc,l_shortcut])
    l_branch = (Activation('relu'))(l_branch)
    
    if j==4:
        l_out1=(Conv2D(10, kernel_size=1,activation='softmax'))(l_branch)
        
    if j==6:
        l_out2=(Conv2D(10, kernel_size=1,activation='softmax'))(l_branch)
        
l_out3=(Conv2D(10, kernel_size=1,activation='softmax'))(l_branch)

#compile model
res_net_short = Model(inputs=input_layer, outputs=[l_out1]) #short model
res_net_short.compile(optimizer=optimizer, loss=losses.categorical_crossentropy)
res_net_short.summary()

res_net_medium = Model(inputs=input_layer, outputs=[l_out2,l_out1]) #medium model
res_net_medium.compile(optimizer=optimizer, loss=losses.categorical_crossentropy)
res_net_medium.summary()

res_net_long = Model(inputs=input_layer, outputs=[l_out3,l_out2,l_out1]) #long model
res_net_long.compile(optimizer=optimizer, loss=losses.categorical_crossentropy)
res_net_long.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 9, 9, 1)      0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 9, 9, 256)    2560        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 9, 9, 128)    295040      conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 9, 9, 128)    295040      conv2d_1[0][0]                   
__________________________________________________________________________________________________
concatenat

In [9]:
%%time
nb_epoch=5
batch_size = 100
val_split = 0.1
history = res_net_short.fit(x, [l_oh], batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_split=val_split)


Train on 900000 samples, validate on 100000 samples
Epoch 1/5
900000/900000 [==============================] - 245s 273us/step - loss: 0.7369 - val_loss: 0.3888
Epoch 2/5
900000/900000 [==============================] - 246s 273us/step - loss: 0.5973 - val_loss: 0.5734
Epoch 3/5
900000/900000 [==============================] - 247s 274us/step - loss: 0.5296 - val_loss: 0.4577
Epoch 4/5
900000/900000 [==============================] - 249s 277us/step - loss: 0.3987 - val_loss: 0.3565
Epoch 5/5
900000/900000 [==============================] - 247s 275us/step - loss: 0.3312 - val_loss: 0.3089
CPU times: user 15min 2s, sys: 3min 50s, total: 18min 53s
Wall time: 20min 35s


In [10]:
%%time
nb_epoch=10
batch_size = 100     
val_split=0.1

history = res_net_medium.fit(x, [l_oh, l_oh], batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_split=val_split)

Train on 900000 samples, validate on 100000 samples
Epoch 1/10
900000/900000 [==============================] - 345s 383us/step - loss: 1.0561 - conv2d_17_loss: 0.6136 - conv2d_12_loss: 0.4425 - val_loss: 0.7992 - val_conv2d_17_loss: 0.4712 - val_conv2d_12_loss: 0.3280
Epoch 2/10
900000/900000 [==============================] - 343s 381us/step - loss: 0.7196 - conv2d_17_loss: 0.4092 - conv2d_12_loss: 0.3104 - val_loss: 0.6271 - val_conv2d_17_loss: 0.3381 - val_conv2d_12_loss: 0.2891
Epoch 3/10
900000/900000 [==============================] - 344s 382us/step - loss: 0.5980 - conv2d_17_loss: 0.3175 - conv2d_12_loss: 0.2805 - val_loss: 0.5703 - val_conv2d_17_loss: 0.2997 - val_conv2d_12_loss: 0.2706
Epoch 4/10
900000/900000 [==============================] - 346s 385us/step - loss: 0.5487 - conv2d_17_loss: 0.2835 - conv2d_12_loss: 0.2651 - val_loss: 0.5285 - val_conv2d_17_loss: 0.2707 - val_conv2d_12_loss: 0.2578
Epoch 5/10
900000/900000 [==============================] - 349s 387us/step 

In [11]:
%%time
val_split=0.1
nb_epoch=10
batch_size = 500     
history = res_net_medium.fit(x, [l_oh, l_oh], batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_split=val_split)
 

Train on 900000 samples, validate on 100000 samples
Epoch 1/10
900000/900000 [==============================] - 250s 277us/step - loss: 0.3585 - conv2d_17_loss: 0.1594 - conv2d_12_loss: 0.1991 - val_loss: 0.3605 - val_conv2d_17_loss: 0.1613 - val_conv2d_12_loss: 0.1992
Epoch 2/10
900000/900000 [==============================] - 250s 278us/step - loss: 0.3560 - conv2d_17_loss: 0.1581 - conv2d_12_loss: 0.1979 - val_loss: 0.3589 - val_conv2d_17_loss: 0.1600 - val_conv2d_12_loss: 0.1989
Epoch 3/10
900000/900000 [==============================] - 249s 277us/step - loss: 0.3531 - conv2d_17_loss: 0.1564 - conv2d_12_loss: 0.1967 - val_loss: 0.3533 - val_conv2d_17_loss: 0.1564 - val_conv2d_12_loss: 0.1968
Epoch 4/10
900000/900000 [==============================] - 249s 277us/step - loss: 0.3498 - conv2d_17_loss: 0.1545 - conv2d_12_loss: 0.1953 - val_loss: 0.3486 - val_conv2d_17_loss: 0.1543 - val_conv2d_12_loss: 0.1943
Epoch 5/10
900000/900000 [==============================] - 249s 277us/step 

In [12]:
%%time
val_split = 0.1
#train long net
nb_epoch=10
batch_size = 100  
history = res_net_long.fit(x, [l_oh,l_oh,l_oh], batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_split=val_split)
 
#seconds round with larger batch-size
nb_epoch=10
batch_size = 500     
history = res_net_long.fit(x, [l_oh,l_oh,l_oh], batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_split=val_split)


Train on 900000 samples, validate on 100000 samples
Epoch 1/10
900000/900000 [==============================] - 399s 443us/step - loss: 0.5501 - conv2d_20_loss: 0.1388 - conv2d_17_loss: 0.1683 - conv2d_12_loss: 0.2430 - val_loss: 0.3755 - val_conv2d_20_loss: 0.0619 - val_conv2d_17_loss: 0.1079 - val_conv2d_12_loss: 0.2057
Epoch 2/10
900000/900000 [==============================] - 402s 447us/step - loss: 0.3497 - conv2d_20_loss: 0.0542 - conv2d_17_loss: 0.0962 - conv2d_12_loss: 0.1993 - val_loss: 0.3398 - val_conv2d_20_loss: 0.0515 - val_conv2d_17_loss: 0.0920 - val_conv2d_12_loss: 0.1963
Epoch 3/10
900000/900000 [==============================] - 400s 444us/step - loss: 0.3296 - conv2d_20_loss: 0.0480 - conv2d_17_loss: 0.0880 - conv2d_12_loss: 0.1936 - val_loss: 0.3180 - val_conv2d_20_loss: 0.0451 - val_conv2d_17_loss: 0.0833 - val_conv2d_12_loss: 0.1897
Epoch 4/10
900000/900000 [==============================] - 402s 447us/step - loss: 0.3163 - conv2d_20_loss: 0.0444 - conv2d_17_loss

In [16]:
%%time
val_split = 0.1
nb_epoch=5
batch_size = 100
val_split = 0.1
history = res_net_short.fit(x, [l_oh], batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_split=val_split)

nb_epoch=10
batch_size = 500     
history = res_net_long.fit(x, [l_oh,l_oh,l_oh], batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_split=val_split)

Train on 900000 samples, validate on 100000 samples
Epoch 1/5
900000/900000 [==============================] - 248s 276us/step - loss: 0.1582 - val_loss: 0.1555
Epoch 2/5
900000/900000 [==============================] - 258s 287us/step - loss: 0.1527 - val_loss: 0.1508
Epoch 3/5
900000/900000 [==============================] - 253s 281us/step - loss: 0.1499 - val_loss: 0.1484
Epoch 4/5
900000/900000 [==============================] - 249s 277us/step - loss: 0.1473 - val_loss: 0.1462
Epoch 5/5
900000/900000 [==============================] - 259s 288us/step - loss: 0.1447 - val_loss: 0.1438
Train on 900000 samples, validate on 100000 samples
Epoch 1/10
900000/900000 [==============================] - 295s 328us/step - loss: 0.2667 - conv2d_20_loss: 0.0406 - conv2d_17_loss: 0.0775 - conv2d_12_loss: 0.1486 - val_loss: 0.2383 - val_conv2d_20_loss: 0.0316 - val_conv2d_17_loss: 0.0635 - val_conv2d_12_loss: 0.1432
Epoch 2/10
900000/900000 [==============================] - 288s 320us/step - l

In [21]:
%%time
val_split=0.1
nb_epoch=10
batch_size = 500     
history = res_net_long.fit(x, [l_oh,l_oh,l_oh], batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_split=val_split)

Train on 900000 samples, validate on 100000 samples
Epoch 1/10
900000/900000 [==============================] - 290s 322us/step - loss: 0.2167 - conv2d_20_loss: 0.0243 - conv2d_17_loss: 0.0532 - conv2d_12_loss: 0.1393 - val_loss: 0.2225 - val_conv2d_20_loss: 0.0262 - val_conv2d_17_loss: 0.0557 - val_conv2d_12_loss: 0.1406
Epoch 2/10
900000/900000 [==============================] - 290s 322us/step - loss: 0.2160 - conv2d_20_loss: 0.0241 - conv2d_17_loss: 0.0529 - conv2d_12_loss: 0.1390 - val_loss: 0.2283 - val_conv2d_20_loss: 0.0272 - val_conv2d_17_loss: 0.0570 - val_conv2d_12_loss: 0.1441
Epoch 3/10
900000/900000 [==============================] - 290s 322us/step - loss: 0.2182 - conv2d_20_loss: 0.0246 - conv2d_17_loss: 0.0535 - conv2d_12_loss: 0.1401 - val_loss: 0.2241 - val_conv2d_20_loss: 0.0264 - val_conv2d_17_loss: 0.0562 - val_conv2d_12_loss: 0.1414
Epoch 4/10
900000/900000 [==============================] - 290s 322us/step - loss: 0.2142 - conv2d_20_loss: 0.0237 - conv2d_17_loss

In [26]:
res_net_long.save('resnetlong3.h5')

In [22]:
def countIncorrect(x,y):
    retVal=0
    for i in range(0,9):
        for j in range(0,9):
            if x[i,j]!=y[i,j]:
                retVal+=1
    return retVal

def countCorrect(x,y):
    retVal=0
    for i in range(0,9):
        for j in range(0,9):
            if x[i,j]==y[i,j]:
                retVal+=1
    return retVal
def countGiven(x):
    retVal=0
    for i in range(0,9):
        for j in range(0,9):
            if x[i,j]!=0:
                retVal+=1
    return retVal
#one hot representation to integer
def OH2int(p):
    retVal=np.zeros((9,9))
    for i in range(0,9):
        for j in range(0,9):
            retVal[i,j]=np.argmax(p[0,i,j,:])
    return retVal

def prColor(text,color):    
    CSI="\x1B["
    reset=CSI+"m"
    print (CSI+color+ text +' '+ CSI + "0m", end='')
    
def prRed(text):      
    red="31;0m" 
    blue="33;36m" 
    green="32;39m" 
    prColor(text,red)

def prGreen(text):      
    red="31;0m" 
    blue="33;36m" 
    green="32;39m" 
    prColor(text,green)

def prBlue(text):      
    red="31;0m" 
    blue="33;36m" 
    green="32;39m" 
    prColor(text,blue)

In [23]:
#evaluate a single sudoku
n=1330

t=x[n,:]
t=t.reshape(-1,9,9,1)


t_label=l[n,:]
t_label=t_label.reshape(-1,9,9,1)
#t_label=t_label.reshape(-1,9,9)



p=res_net_long.predict(t)
p=np.array(p)
p=p[0,:,:,:,:]
p=OH2int(p)

print('Corect symbols:', countCorrect(np.round(p.reshape(9,9)),t_label.reshape(9,9)), '/ 81')
print('Given symbols:', countGiven(t.reshape(9,9)), '/ 81')
guessing=round((81-countGiven(t.reshape(9,9)))/9)
print('Guessing:', countGiven(t.reshape(9,9))+guessing, '/ 81')

#print('metrics [mse/ber/bler]:', model.evaluate(t, t_label,verbose=0),'\n')
print('Input Sudoku:\n\n', t.reshape(9,9), '\n')
print('Sudoku after CNN:\n\n', np.round(p.reshape(9,9)),'\n')
print('Correct solution:\n\n', np.round(t_label.reshape(9,9)),'\n')

#print Sudoku
p=p.reshape(9,9)
t_label=t_label.reshape(9,9)
t=t.reshape(9,9)
for i in range(0,9):
    for j in range(0,9):
        if t[i,j]==0:
            if p[i,j]==t_label[i,j]:
                prGreen(str(int(p[i,j])))
            else:
                prRed(str(int(p[i,j])))
        else:
            prBlue(str(int(p[i,j])))
            
    print()

Corect symbols: 81 / 81
Given symbols: 34 / 81
Guessing: 39 / 81
Input Sudoku:

 [[0 0 1 9 0 0 3 0 0]
 [0 9 0 0 2 7 0 6 0]
 [0 0 6 5 1 0 0 0 4]
 [0 0 8 0 0 3 7 0 0]
 [5 4 0 0 7 6 0 2 0]
 [3 0 0 1 0 0 0 4 0]
 [0 8 0 0 0 0 0 0 5]
 [0 1 0 7 9 2 8 0 0]
 [6 7 0 4 0 5 9 0 0]] 

Sudoku after CNN:

 [[ 2.  5.  1.  9.  6.  4.  3.  8.  7.]
 [ 8.  9.  4.  3.  2.  7.  5.  6.  1.]
 [ 7.  3.  6.  5.  1.  8.  2.  9.  4.]
 [ 1.  6.  8.  2.  4.  3.  7.  5.  9.]
 [ 5.  4.  9.  8.  7.  6.  1.  2.  3.]
 [ 3.  2.  7.  1.  5.  9.  6.  4.  8.]
 [ 9.  8.  2.  6.  3.  1.  4.  7.  5.]
 [ 4.  1.  5.  7.  9.  2.  8.  3.  6.]
 [ 6.  7.  3.  4.  8.  5.  9.  1.  2.]] 

Correct solution:

 [[2 5 1 9 6 4 3 8 7]
 [8 9 4 3 2 7 5 6 1]
 [7 3 6 5 1 8 2 9 4]
 [1 6 8 2 4 3 7 5 9]
 [5 4 9 8 7 6 1 2 3]
 [3 2 7 1 5 9 6 4 8]
 [9 8 2 6 3 1 4 7 5]
 [4 1 5 7 9 2 8 3 6]
 [6 7 3 4 8 5 9 1 2]] 

2 5 1 9 6 4 3 8 7 
8 9 4 3 2 7 5 6 1 
7 3 6 5 1 8 2 9 4 
1 6 8 2 4 3 7 5 9 
5 4 9 8 7 6 1 2 3 
3 2 7 1 5 9 6 4 8 
9 8 2 6 3 1 4 7 5 
4 1 5 7 

In [24]:
#evaluate all Sudokus at once
N=1000

res_correct=np.zeros((N,1))
res_guessing=np.zeros((N,1))
completly_solved=0

givenFields=np.zeros((81,1))
solvedFields=np.zeros((81,1))

offset=900000#use the unused Sudokus for evaluation

for n in range (0,N):

    t=x[n+offset,:]
    t=t.reshape(-1,9,9,1)
    t_label=l[n+offset,:]
    t_label=t_label.reshape(-1,9,9,1)

    p=res_net_long.predict(t)
    p=np.array(p)
    p=p[0,:,:,:,:]
    p=OH2int(p)

    res_correct[n]=countCorrect(np.round(p.reshape(9,9)),t_label.reshape(9,9))
    res_guessing[n]=countGiven(t.reshape(9,9))+round((81-countGiven(t.reshape(9,9)))/9)
    
    temp=countGiven(t.reshape(9,9))+round((81-countGiven(t.reshape(9,9)))/9)
    
    givenFields[temp]+=1
    
    
    if res_correct[n]==81:
        completly_solved+=1
        solvedFields[temp]+=1

diff=(res_correct-res_guessing)
print('In average NN solves:', np.mean(res_correct), '/ 81')
print('In average NN solves more than Guessing:', np.mean(diff), '/ 81')
print('Completly solved Sudokus: ', completly_solved)


In average NN solves: 80.27 / 81
In average NN solves more than Guessing: 41.382 / 81
Completly solved Sudokus:  620


In [ ]:
from keras.models import load_model
res_net_long = load_model('resnetlong2.h5')


In [ ]:
#plot results
import matplotlib.pyplot as plt
#plt.plot(diff)
plt.plot(res_correct)
plt.show()